In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("/content/crime_data.csv")

1.1 Reading the data

In [3]:
df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,241711715,08/01/2024 12:00:00 AM,08/01/2024 12:00:00 AM,1319,17,Devonshire,1791,1,440,THEFT PLAIN - PETTY ($950 & UNDER),...,IC,Invest Cont,440.0,NaN,NaN,NaN,8300 KELVIN AV,NaN,34.2200,-118.5863
1,231014031,09/21/2023 12:00:00 AM,09/15/2023 12:00:00 AM,1930,10,West Valley,1011,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,18900 CANTLAY ST,NaN,34.2023,-118.5458
2,231010808,06/27/2023 12:00:00 AM,06/26/2023 12:00:00 AM,1230,10,West Valley,1015,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,7300 ENFIELD AV,NaN,34.2033,-118.5241
3,211410441,04/25/2021 12:00:00 AM,04/25/2021 12:00:00 AM,2330,14,Pacific,1488,2,626,INTIMATE PARTNER - SIMPLE ASSAULT,...,IC,Invest Cont,626.0,NaN,NaN,NaN,5800 W CENTURY BL,NaN,33.9456,-118.3835
4,211114569,10/25/2021 12:00:00 AM,10/25/2021 12:00:00 AM,1455,11,Northeast,1123,1,210,ROBBERY,...,IC,Invest Cont,210.0,NaN,NaN,NaN,2900 LOS FELIZ BL,NaN,0.0000,0.0000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32357 entries, 0 to 32356
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   DR_NO           32357 non-null  int64  
 1   Date Rptd       32357 non-null  object 
 2   DATE OCC        32357 non-null  object 
 3   TIME OCC        32357 non-null  int64  
 4   AREA            32357 non-null  int64  
 5   AREA NAME       32357 non-null  object 
 6   Rpt Dist No     32357 non-null  int64  
 7   Part 1-2        32357 non-null  int64  
 8   Crm Cd          32357 non-null  int64  
 9   Crm Cd Desc     32357 non-null  object 
 10  Mocodes         27483 non-null  object 
 11  Vict Age        32357 non-null  int64  
 12  Vict Sex        27702 non-null  object 
 13  Vict Descent    27701 non-null  object 
 14  Premis Cd       32357 non-null  float64
 15  Premis Desc     32342 non-null  object 
 16  Weapon Used Cd  10544 non-null  float64
 17  Weapon Desc     10544 non-null 

1.2. Identifying missing values and dropping the ones which have missing over 50%

In [6]:
total_missing = df.isna().sum()
total_missing

,0
DR_NO,0
Date Rptd,0
DATE OCC,0
TIME OCC,0
AREA,0
AREA NAME,0
Rpt Dist No,0
Part 1-2,0
Crm Cd,0
Crm Cd Desc,0


In [7]:
missing_over_50 = total_missing[total_missing > (0.5 * len(df))].index
df_cleaned = df.drop(columns=missing_over_50)
print("Dropped cols:", missing_over_50)

Dropped cols: Index(['Weapon Used Cd', 'Weapon Desc', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4',
       'Cross Street'],
      dtype='object')


1.3. Converting to datetime, creating year, month, day and hour cols

In [8]:
df_cleaned['DATE OCC'] = pd.to_datetime(df_cleaned['DATE OCC'])
df_cleaned['Year'] = df_cleaned['DATE OCC'].dt.year
df_cleaned['Month'] = df_cleaned['DATE OCC'].dt.month
df_cleaned['Day'] = df_cleaned['DATE OCC'].dt.day

<ipython-input-8-d5ecf8c3a38a>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_cleaned['DATE OCC'] = pd.to_datetime(df_cleaned['DATE OCC'])


In [10]:
df_cleaned[['DATE OCC', 'Year', 'Month', 'Day']].head()

,DATE OCC,Year,Month,Day
0,2024-08-01,2024,8,1
1,2023-09-15,2023,9,15
2,2023-06-26,2023,6,26
3,2021-04-25,2021,4,25
4,2021-10-25,2021,10,25


In [11]:
df_cleaned['Hour'] = df_cleaned['TIME OCC'].astype(str).str.zfill(4).str[:2].astype(int)

In [13]:
df_cleaned[['TIME OCC', 'Hour']].head()

,TIME OCC,Hour
0,1319,13
1,1930,19
2,1230,12
3,2330,23
4,1455,14


1.4. Filtering the data to display instances for which year is 2023 and crime is burglary.

In [22]:
df_cleaned[(df_cleaned['Crm Cd Desc'].str.contains('BURGLARY')) & (df_cleaned['Year']==2023)]

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,LOCATION,LAT,LON,Year,Month,Day,Hour
47,231107877,04/15/2023 12:00:00 AM,2023-01-15,500,11,Northeast,1151,1,310,BURGLARY,...,IC,Invest Cont,310.0,5000 W SUNSET BL,34.0981,-118.2983,2023,1,15,5
53,231312734,06/23/2023 12:00:00 AM,2023-06-23,1100,13,Newton,1331,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,3200 S GRAND AV,34.0216,-118.2754,2023,6,23,11
130,241305432,01/31/2024 12:00:00 AM,2023-07-31,2130,13,Newton,1385,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,5800 S CENTRAL AV,33.9857,-118.2628,2023,7,31,21
147,231912840,08/15/2023 12:00:00 AM,2023-08-14,2200,19,Mission,1962,1,310,BURGLARY,...,IC,Invest Cont,310.0,15000 CORE LN,34.2424,-118.4596,2023,8,14,22
158,230817252,11/21/2023 12:00:00 AM,2023-11-21,1545,8,West LA,802,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,WILL ROGERS STATE PARK,34.0506,-118.5131,2023,11,21,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32045,230914151,09/18/2023 12:00:00 AM,2023-09-17,505,9,Van Nuys,974,1,310,BURGLARY,...,IC,Invest Cont,310.0,4700 VAN NUYS BL,34.1560,-118.4487,2023,9,17,5
32122,230607116,03/12/2023 12:00:00 AM,2023-03-12,800,6,Hollywood,659,1,310,BURGLARY,...,IC,Invest Cont,310.0,5400 FOUNTAIN AV,34.0966,-118.3049,2023,3,12,8
32142,230401591,11/13/2023 12:00:00 AM,2023-11-10,1600,4,Hollenbeck,497,1,310,BURGLARY,...,IC,Invest Cont,310.0,1700 INDUSTRIAL WY,34.0141,-118.2069,2023,11,10,16
32277,230816354,10/30/2023 12:00:00 AM,2023-10-29,2300,8,West LA,889,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,PREUSS RD,34.0408,-118.3864,2023,10,29,23


1.5. Grouping by area name and displaying total number of crimes for each and average victim age

Showing in descending order

In [23]:
grouped = df_cleaned.groupby('AREA NAME').agg(
    Total_Crimes=('Crm Cd Desc', 'count'),
    Avg_Victim_Age=('Vict Age', 'mean')
).sort_values(by='Total_Crimes', ascending=False)

In [24]:
grouped

,Total_Crimes,Avg_Victim_Age
AREA NAME,,
Central,2279,28.602457
77th Street,2024,28.809289
Pacific,1960,28.646939
Southwest,1843,28.816061
Hollywood,1684,28.840261
Newton,1655,25.089426
N Hollywood,1638,27.827839
Olympic,1602,28.003121
Southeast,1564,28.312020


2.1. finding 3 most frequent crimes

In [25]:
top_crimes = df['Crm Cd Desc'].value_counts()

In [26]:
top_crimes.head(3)

,count
Crm Cd Desc,
VEHICLE - STOLEN,3742
BATTERY - SIMPLE ASSAULT,2344
BURGLARY FROM VEHICLE,2079


2.2. grouping by Hour and counting number of crimes at each hour

In [27]:
df_cleaned.groupby('Hour').value_counts().count()

27460

In [28]:
hourly_crimes = df_cleaned.groupby('Hour').size()

In [29]:
hourly_crimes

,0
Hour,
0,1283
1,957
2,803
3,680
4,620
5,549
6,701
7,842
8,1154


2.3. grouping by victim sex and calculating their average age and total crimes

In [30]:
victim_stats = df_cleaned.groupby('Vict Sex').agg(
    Total_Crimes=('Crm Cd Desc', 'count'),
    Avg_Victim_Age=('Vict Age', 'mean')
)
print("Victim Statistics by Sex:\n", victim_stats)

Victim Statistics by Sex:
           Total_Crimes  Avg_Victim_Age
Vict Sex                              
F                11585       38.229003
H                    2       25.000000
M                12988       37.144056
X                 3127        2.691717


4.1. creating severity score with 5 pts for weapon, 3 pts for burglary and all others of 1 point

In [ ]:
df_cleaned.columns

Index(['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME',
       'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Mocodes',
       'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd', 'Premis Desc',
       'Status', 'Status Desc', 'Crm Cd 1', 'LOCATION', 'LAT', 'LON', 'Year',
       'Month', 'Day', 'Hour', 'Severity Score'],
      dtype='object')

as df_cleaned already dropped weapon column, we'll go with the original data

In [31]:
df['Severity Score'] = 0
df.loc[df['Weapon Used Cd'].notna(), 'Severity Score'] += 5
df.loc[df['Crm Cd Desc'] == 'BURGLARY', 'Severity Score'] += 3
df.loc[df['Severity Score'] == 0, 'Severity Score'] = 1

In [32]:
df[['Weapon Used Cd', 'Crm Cd Desc', 'Severity Score']]

,Weapon Used Cd,Crm Cd Desc,Severity Score
0,NaN,THEFT PLAIN - PETTY ($950 & UNDER),1
1,NaN,THEFT OF IDENTITY,1
2,NaN,THEFT OF IDENTITY,1
3,400.0,INTIMATE PARTNER - SIMPLE ASSAULT,5
4,200.0,ROBBERY,5
...,...,...,...
32352,510.0,BATTERY - SIMPLE ASSAULT,5
32353,NaN,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",1
32354,NaN,THEFT OF IDENTITY,1
32355,NaN,THEFT OF IDENTITY,1


grouping by area name and displaying severity score for each

In [33]:
score_by_area = df.groupby('AREA NAME')['Severity Score'].sum()

In [34]:
score_by_area

,Severity Score
AREA NAME,
77th Street,6094
Central,5547
Devonshire,2898
Foothill,2486
Harbor,3144
Hollenbeck,2957
Hollywood,4433
Mission,2977
N Hollywood,3554


bonus part

printing the values in bounding box of a specific latitude and longtitude values

In [35]:
lat_min, lat_max = 34.0400, 34.0600  # example values
lon_min, lon_max = -118.2700, -118.2400
downtown_crimes = df_cleaned[(df_cleaned['LAT'] >= lat_min) & (df_cleaned['LAT'] <= lat_max) &
                             (df_cleaned['LON'] >= lon_min) & (df_cleaned['LON'] <= lon_max)]
print("Crimes in the specified bounding box:\n", downtown_crimes)

Crimes in the specified bounding box:
            DR_NO               Date Rptd   DATE OCC  TIME OCC  AREA AREA NAME  \
10     200213278  08/02/2020 12:00:00 AM 2020-08-01       700     2   Rampart   
16     240111628  05/20/2024 12:00:00 AM 2024-05-19      1850     1   Central   
25     220120317  09/12/2022 12:00:00 AM 2022-08-23      1200     1   Central   
31     210212596  07/29/2021 12:00:00 AM 2021-07-29      1120     2   Rampart   
61     200120593  11/04/2020 12:00:00 AM 2020-11-02      2030     1   Central   
...          ...                     ...        ...       ...   ...       ...   
32268  200110079  04/03/2020 12:00:00 AM 2020-04-03       615     1   Central   
32275  210118618  10/14/2021 12:00:00 AM 2021-10-14       950     1   Central   
32328  220115690  07/06/2022 12:00:00 AM 2022-07-05      2000     1   Central   
32337  210117420  09/24/2021 12:00:00 AM 2021-09-24       115     1   Central   
32353  200207823  03/16/2020 12:00:00 AM 2020-03-14      1012     2   

we got 1841 instances in this specific bounding box